In [111]:
import pandas as pd
import requests

In [112]:
from yahooquery import Ticker

cvs = Ticker('cvs')

data = cvs.summary_detail['cvs']
data

{'maxAge': 1,
 'priceHint': 2,
 'previousClose': 75.75,
 'open': 75.48,
 'dayLow': 74.97,
 'dayHigh': 75.59,
 'regularMarketPreviousClose': 75.75,
 'regularMarketOpen': 75.48,
 'regularMarketDayLow': 74.97,
 'regularMarketDayHigh': 75.59,
 'dividendRate': 2.0,
 'dividendYield': 0.026600001,
 'exDividendDate': '2021-04-21 20:00:00',
 'payoutRatio': 0.36560002,
 'fiveYearAvgDividendYield': 2.78,
 'beta': 0.821842,
 'trailingPE': 13.748857,
 'forwardPE': 9.27284,
 'volume': 3768110,
 'regularMarketVolume': 3768110,
 'averageVolume': 6756567,
 'averageVolume10days': 5999733,
 'averageDailyVolume10Day': 5999733,
 'bid': 0.0,
 'ask': 0.0,
 'bidSize': 900,
 'askSize': 1000,
 'marketCap': 98583371776,
 'fiftyTwoWeekLow': 55.36,
 'fiftyTwoWeekHigh': 77.44,
 'priceToSalesTrailing12Months': 0.36797473,
 'fiftyDayAverage': 74.588234,
 'twoHundredDayAverage': 70.16089,
 'trailingAnnualDividendRate': 2.0,
 'trailingAnnualDividendYield': 0.026402641,
 'currency': 'USD',
 'fromCurrency': None,
 'toCur

In [113]:
# Question 1 for PE and Stock Price
print(data['previousClose'])
print(data['trailingPE'])

75.75
13.748857


In [114]:
# Question 2 plowback ratio
print(1-data['payoutRatio'])

0.63439998


In [115]:
# Question 3 5 year growth estimate
import re
import json

r = requests.get("https://finance.yahoo.com/quote/CVS/analysis?p=CVS")

pull = json.loads(re.search('root\.App\.main\s*=\s*(.*);', r.text).group(1))

field = [t for t in pull["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["earningsTrend"]["trend"] if t["period"] == "+5y" ][0]

print("Next 5 Years (per annum) : " + field["growth"]["fmt"])

Next 5 Years (per annum) : 3.77%


In [116]:
# Question 4 CAPM
RiskFree = 0.01
MarketRiskPremium = 0.06
capm = RiskFree+data['beta']*MarketRiskPremium
capm = "{:.3%}".format(capm)
print(capm)

5.931%


In [117]:
# Question 5 on GGM
# in GGM ((1-b)(1+g))/r-g=pe

def p2f(x):
    return float(x.strip('%'))/100

g = p2f(field["growth"]["fmt"])
b = 1-data['payoutRatio']
r = p2f(capm)
print(g,b,r)

GordonGrowthModelPE = ((1+g)*(1-b))/(r-g)
GordonGrowthModelPE = "{0:.2f}".format(GordonGrowthModelPE)
print("The GGM PE is: "+str(GordonGrowthModelPE))

0.0377 0.63439998 0.05931
The GGM PE is: 17.56


In [118]:
#Question 6 What growth rate does the current market price-earnings ratio imply, under our numerical assumptions regarding the value of the risk-free rate and the market risk premium.
g = 0.0
PEratio = data['trailingPE']
g = abs((((1+g)*(1-b))/PEratio)-r)
g = "{:.3%}".format(g)

In [119]:
g

'3.272%'